# Obter dados no drive

In [ ]:
!pip install pyxlsb
!pip install -q git+https://github.com/tensorflow/examples.git

In [ ]:
from google.colab import drive
import pandas as pd
from pyxlsb import open_workbook
import random

import zipfile
import os

from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

from glob import glob
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow_examples.models.pix2pix import pix2pix
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
drive.mount('/content/drive')

# Divisão de imagens e masks em amostras de treino e teste

In [ ]:
images = []
masks = []

for path in glob('/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/cropped_images/*/*'):
  images.append(path + '/image.tif')
  masks.append(path + '/mask.png')

In [ ]:
# Função para carregar e pré-processar uma imagem e sua máscara
def load_and_preprocess_image(image_path, mask_path, target_size):

    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image) / 255.0  # Normalização entre 0 e 1

    mask = load_img(mask_path, target_size=target_size, color_mode='grayscale')
    mask = img_to_array(mask) / 255.0  # Normalização entre 0 e 1

    return image, mask

In [ ]:
# Lista para armazenar imagens e máscaras pré-processadas
images_processed = []
masks_processed = []

count = 1
# Carregar e pré-processar todas as imagens e máscaras
for img_path, mask_path in zip(images, masks):
    print(count)
    count += 1
    img, mask = load_and_preprocess_image(img_path, mask_path, target_size=(256, 256))
    images_processed.append(img)
    masks_processed.append(mask)

# Converter para arrays numpy
images_processed = np.array(images_processed)
masks_processed = np.array(masks_processed)


In [ ]:
images_processed

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(images_processed, masks_processed, test_size=0.2, random_state=42)

# Definição do modelo

In [ ]:
def Unet(input_shape):
    inputs = tf.keras.Input(shape=input_shape)

    # Encoder (contraction path)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    # Decoder (expansion path)
    up3 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv2)
    merge3 = layers.concatenate([conv1, up3], axis=3)
    conv3 = layers.Conv2D(64, 3, activation='relu', padding='same')(merge3)
    conv3 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv3)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv3)  # Saída com um canal (máscara binária)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
# Criar modelo U-Net
model = Unet(input_shape=(256, 256, 3))
#model = UNet(output_channels=1)
model.summary()

In [ ]:
#forma de fazer com que o treino pare após obter uma acurácia específica
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if 'accuracy' in logs and logs.get('accuracy') is not None and logs.get('accuracy') > 0.95:
            print("\nAtingimos uma precisão maior que 95.0%, então podemos parar o treinamento!")
            self.model.stop_training = True


In [ ]:
# Função para calcular o Dice Coefficient
def dice_coefficient(y_train, y_val):
    smooth = 1e-6
    intersection = tf.reduce_sum(y_train * y_val)
    dice_coefficient = (2. * intersection + smooth) / (tf.reduce_sum(y_train) + tf.reduce_sum(y_val) + smooth)
    return dice_coefficient

# Função de perda de Dice
def dice_loss(y_train, y_val):
    return 1 - dice_coefficient(y_train, y_val)

# Função para calcular a penalidade adicional
def penalty_loss(y_train, y_val, penalty_weight):
    # Calcular a penalidade considerando a diferença entre y_train e y_val
    penalty = tf.reduce_sum(tf.abs(y_train - y_val))
    # Multiplicar a penalidade pelo peso da penalidade
    weighted_penalty = penalty_weight * penalty
    return weighted_penalty

# Função de perda combinada
def combined_loss(y_train, y_val, penalty_weight):
    # Perda padrão (por exemplo, perda de entropia cruzada binária)
    standard_loss = tf.keras.losses.binary_crossentropy(y_train, y_val)
    # Dice Loss
    dice = dice_loss(y_train, y_val)
    # Penalidade adicional
    penalty = penalty_loss(y_train, y_val, penalty_weight)
    # Perda total = perda padrão + penalidade + Dice Loss
    total_loss = standard_loss + penalty + dice
    return total_loss

# Métrica de acurácia customizada
def custom_accuracy(y_train, y_val):
    # Calcular a acurácia considerando uma tolerância de 0.5 na predição
    y_val_binary = tf.round(y_val)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(y_train, y_val_binary), tf.float32))
    return accuracy

# Função para cálculo do Coverage Rate
def coverage_rate(y_true, y_pred, threshold=0.5):
    # Binarize the predictions
    y_pred = tf.where(y_pred >= threshold, 1.0, 0.0)

    # Calculate the coverage rate
    intersection = tf.reduce_sum(y_true * y_pred)
    total_foreground = tf.reduce_sum(y_true)

    coverage = intersection / (total_foreground + tf.keras.backend.epsilon())

    return coverage

# Treino do modelo

In [ ]:
callbacks = myCallback()

# Compilar o modelo
model.compile(optimizer='adam', loss=lambda y_train, y_val: combined_loss(y_train, y_val, 0.001), metrics=[custom_accuracy, coverage_rate])
with tf.device('/gpu:0'):
  # Treinar o modelo
  H = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=200, batch_size=16, callbacks=callbacks)

# Avaliação do modelo

In [ ]:
with tf.device('/gpu:0'):
  # Avaliando o modelo
  loss, accuracy, coverage = model.evaluate(X_val, y_val)
  print(f'Acurácia do modelo: {accuracy}')

  # Mostrando resultados
  plt.style.use("ggplot")
  plt.figure()
  plt.plot(H.epoch, H.history["loss"], label="train_loss")
  plt.plot(H.epoch, H.history["val_loss"], label="val_loss")
  plt.plot(H.epoch, H.history["custom_accuracy"], label="train_acc")
  plt.plot(H.epoch, H.history["val_custom_accuracy"], label="val_acc")
  plt.title("Training Loss and Accuracy")
  plt.xlabel("Epoch #")
  plt.ylabel("Loss/Accuracy")
  plt.legend()
  plt.show()

In [ ]:
#  ZNB\

In [ ]:
with tf.device('/gpu:0'):
  # Avaliar o modelo nos dados de teste
  results = model.evaluate(X_val, y_val)
  print("Test Loss:", results[0])
  print("Test Accuracy:", results[1])
  print("Test Coverage Rate:", results[2])

  # Prever máscaras usando o modelo
  predicted_masks = model.predict(X_val)

In [ ]:
with tf.device('/gpu:0'):
  # Obter métricas de precisão e perda do histórico de treinamento
  acc = H.history['custom_accuracy']
  val_acc = H.history['val_custom_accuracy']
  loss = H.history['loss']
  val_loss = H.history['val_loss']

  # Número de épocas
  epochs = range(1, len(acc) + 1)

  # Plotar precisão do conjunto de treino e validação
  plt.plot(epochs, acc, 'r', label='Precisão do Conjunto de Treino')
  plt.plot(epochs, val_acc, 'b', label='Precisão do Conjunto de Validação')
  plt.title('Precisão do Conjunto de Treino e Validação')
  plt.xlabel('Épocas')
  plt.ylabel('Precisão')
  plt.legend()
  plt.show()

  # Plotar perda do conjunto de treino e validação
  plt.plot(epochs, loss, 'r', label='Perda do Conjunto de Treino')
  plt.plot(epochs, val_loss, 'b', label='Perda do Conjunto de Validação')
  plt.title('Perda do Conjunto de Treino e Validação')
  plt.xlabel('Épocas')
  plt.ylabel('Perda')
  plt.legend()
  plt.show()

In [ ]:
with tf.device('/gpu:0'):
    # Gerar as saídas do modelo para um conjunto de entradas de teste
    saidas_modelo = model.predict(X_val)
    imgs_preditas = []

    # Iterar sobre cada saída do modelo
    for i in range(len(X_val)):
        # Obter a entrada correspondente e a saída real
        img_entrada = X_val[i]
        img_saida_real = y_val[i]

        # Obter a saída gerada pelo modelo
        img_saida_modelo = saidas_modelo[i]

        #Aplicar limiarização apenas durante a inferência
        img_saida_modelo_limiarizada = np.where(img_saida_modelo < 0.5, 0, 1)

        imgs_preditas.append(img_saida_modelo_limiarizada)

        # Mostrar as imagens
        plt.figure(figsize=(12, 4))

        plt.subplot(1, 3, 1)
        plt.imshow(img_entrada)
        plt.title('Entrada')
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.imshow(img_saida_real, cmap='gray')
        plt.title('Saída Esperada')
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.imshow(img_saida_modelo_limiarizada, cmap='gray')
        plt.title('Saída do Modelo (Limiarizada)')
        plt.axis('off')

        plt.show()

In [ ]:
iou_scores = []
with tf.device('/gpu:0'):
    for mask, result in zip(y_val, imgs_preditas):
        intersection = np.logical_and(mask, result)
        union = np.logical_or(mask, result)

        iou_score = np.sum(intersection) / np.sum(union)
        iou_scores.append(iou_score)

iou_mean = np.mean(iou_scores)
print('Média dos IoU:', iou_mean)